In [1]:
# ! pip install unstructured
# ! pip install python-magic-bin


In [2]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
import logging
from langchain.vectorstores import FAISS
import faiss
from dotenv import find_dotenv, load_dotenv
import pickle

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
# Load texts from urls
urls = [
  'https://spurshop.com.ng/',
  'https://spurshop.com.ng/dictamni-kenya/',
  'https://spurshop.com.ng/tablet/',
  'https://spurshop.com.ng/dictamni-thanks/'
  ]
loaders = UnstructuredURLLoader(urls=urls)
data = loaders.load()
data

[Document(page_content="Skip to content\n\n(40% Discount + FREE DELIVERY Today)\n\nUltimate\n\n4in1\n\nEssential Teeth\n\nPowder\n\nPrevious\n\nNext\n\nListen, just pay attention and you will see the magic in few days from now\n\nDiscover how to remove toxins/diseases from your teeth, eradicate bad breath/mouth odour, fight tooth diseases and WHITEN your teeth permanently....\n\nYou have tried whitening toothpaste, whitening strips, and even bleaching trays but you are still sporting those dingy, yellowish looking teeth. Are you doing it all wrong? Are your teeth just destined to be dingy?\n\nLet’s look at some things that might keep your teeth whitening treatment from working.\n\nThe Number 1 Reason Dental restorations, such as crowns or veneers, DO NOT respond to tooth whitening products. If you have any of these restorations, especially in your front teeth, they will not whiten with bleaching products.\n\nNumber 2 Reason Is That Those Products Did Not Contain Activated Pearl Powder 

In [5]:
# Split data in chunks
text_splitter = CharacterTextSplitter(
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
)
docs = text_splitter.split_documents(data)

In [6]:
len(docs)

24

In [7]:
docs

[Document(page_content='Skip to content\n\n(40% Discount + FREE DELIVERY Today)\n\nUltimate\n\n4in1\n\nEssential Teeth\n\nPowder\n\nPrevious\n\nNext\n\nListen, just pay attention and you will see the magic in few days from now\n\nDiscover how to remove toxins/diseases from your teeth, eradicate bad breath/mouth odour, fight tooth diseases and WHITEN your teeth permanently....\n\nYou have tried whitening toothpaste, whitening strips, and even bleaching trays but you are still sporting those dingy, yellowish looking teeth. Are you doing it all wrong? Are your teeth just destined to be dingy?\n\nLet’s look at some things that might keep your teeth whitening treatment from working.\n\nThe Number 1 Reason Dental restorations, such as crowns or veneers, DO NOT respond to tooth whitening products. If you have any of these restorations, especially in your front teeth, they will not whiten with bleaching products.', metadata={'source': 'https://spurshop.com.ng/'}),
 Document(page_content='Numbe

In [8]:
# Initialize OpenAI Embeddings
embeddings = OpenAIEmbeddings()

# Store embeddings in vector database
vectorstore = FAISS.from_documents(docs, embeddings)

In [9]:
# Save vectorstore in local storage
with open('website_vector_store.pkl', 'wb') as f:
  pickle.dump(vectorstore, f)

In [10]:
# Load vectorstore
with open('website_vector_store.pkl', 'rb') as f:
  vectorestore = pickle.load(f)

In [11]:

llm = ChatOpenAI(model_name="gpt-3.5-turbo")

In [12]:
question = "Do you cure pile?"

In [13]:
# Generate similar answers to question
logging.basicConfig()
logging.getLogger('langchain.retrievers.multi_query').setLevel(logging.INFO)
retriever_from_llm = MultiQueryRetriever.from_llm(retriever=vectorstore.as_retriever(),
                                                  llm=ChatOpenAI(temperature=0))
unique_docs = retriever_from_llm.get_relevant_documents(query=question)
unique_docs

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the treatment options for piles?', '2. What are the effective remedies for curing piles?', '3. Are there any recommended treatments available for piles?']


[Document(page_content='If the above symptoms sound familiar, you most likely have hemorrhoids (Pile), but you have nothing to be ashamed of or concerned about because it is a condition that affects millions of men and women of all ages, and because you have now found this post, your healing is just waiting for you to take action right away.\n\nPlease Take Action Now By Clicking This Link.\n\nSave Yourself Pains and The Risk of Surgery Later with Unnecessary Expenses With Dictamni Anti-bacterial Cream\n\nDictamni Anti-bacteria Cream has effective anti-inflammatory, astringent and antiseptic herbal properties to stimulate the anal skin healing process. It also helps to restore dilated and inflamed veins to normal conditions.', metadata={'source': 'https://spurshop.com.ng/dictamni-kenya/'}),
 Document(page_content='Skip to content\n\nFREE SHIPPING NATIONWIDE\n\nHeal Pile (Hemorrhoids)\n\nOur hemorrhoids ointment is a proven and effective product that is made with natural ingredients that

In [14]:
# Retrieve answer
qa_chain = RetrievalQA.from_chain_type(llm,retriever=vectorstore.as_retriever())
qa_chain({"query": question})

{'query': 'Do you cure pile?',
 'result': 'Yes, our product, Dictamni Anti-bacterial Cream, is specifically designed to help heal and treat hemorrhoids (piles). It contains natural ingredients that have been used for thousands of years to cure hemorrhoids. It has effective anti-inflammatory, astringent, and antiseptic properties that stimulate the healing process of the anal skin and help restore dilated and inflamed veins to normal conditions.'}

In [15]:
sales_template = """As a sales reprentative, your goal is to convince potential customers to make a purchase using
relevant and helpful information that will calm whatever concerns or reservations they might have about the product.
You should answer the answer customer inquiries based on the context provided and avoid making up answers. If you 
don't know the answer, simply say you don't know while convincing them to try out the product. 
Ask customers what product they want and confirm what country they are in and then offer them prices in their currency 
where KES is for Kenyas Shillings and GHC is for Ghana Cedis. Remember to provide relevant information about product 
features that highlights it's benefits and use cases to assist the user in understanding its value. Keep the answer
as concise as possible.
{context}

Question: {question}
Helpful Answer:
"""

In [16]:
# chain = LLMChain()